# Install the requirements

In [1]:
!pip install -q gradio adapters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.4/260.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.9/315.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.3 MB/s eta 0:0

# Import the libraires

In [2]:
import gradio as gr
import re
from transformers import AutoTokenizer,TextClassificationPipeline
from adapters import AutoAdapterModel

# Pre-Process Function

In [3]:
# This function removes new lines, digits, code and new lines.
def preprocess(issue):
    issue = re.sub(r'```.*?```', ' ', issue, flags=re.DOTALL)
    issue = re.sub(r'\n', ' ', issue)
    issue = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', issue)
    issue = re.sub(r'\d+', ' ', issue)
    issue = re.sub(r'[^a-zA-Z0-9?\s]', ' ', issue)
    issue = re.sub(r'\s+', ' ', issue)
    return issue

# Inference Function

In [4]:
def text_classification(text):
    tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base", max_length=256, truncation=True, padding="max_length")
    model = AutoAdapterModel.from_pretrained("FacebookAI/roberta-base")
    adapter_react = model.load_adapter("buelfhood/irc-single-adapter", source = "hf",set_active=True)
    classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, max_length=256, padding="max_length", truncation=True,top_k=None)
    preprocessed_issue = preprocess (text)
    out = classifier(preprocessed_issue)[0]
    label_scores = {result['label']: result['score'] for result in out}
    return label_scores

# eIRC Webapp

In [7]:
examples=["This is a question", "This is a bug", "This is an enhancement" ]

io = gr.Interface(fn=text_classification,
                         inputs= gr.Textbox(lines=3, label="Text", placeholder="Enter the text of the issue here:"),
                         outputs="label",
                         title="eIRC: An Efficient Issue Report Classification Web Application",
                         description="Enter a text of an issue and see whether it is a bug, feature or question?",
                         examples=examples)

io.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://40e006ff2fc6536d4e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
